# SpeedData Analysis Example

This notebook demonstrates how to use SpeedData's Pivot REST API and Roelle DataSet for data analysis.

## Setup

Add SpeedData library to Python path:

In [ ]:
import sys
sys.path.insert(0, '../../lib/python')

import requests
from dataset import read_hdf5, read_avro
import matplotlib.pyplot as plt
import numpy as np

## Method 1: Load data from Pivot REST API

Query SpeedData Pivot API to transform AVRO data to HDF5 format:

In [ ]:
# Query Pivot API for last 30 seconds of data
response = requests.get('http://localhost:8000/api/v1/pivot/export', params={
    'start': '2025-01-23T14:00:00Z',
    'duration': 'PT30S',
    'channels': 'example'
})

# Save HDF5 file
with open('export.h5', 'wb') as f:
    f.write(response.content)

print(f"Downloaded {len(response.content)} bytes")

# Load into DataSet
data = read_hdf5('export.h5')
print(data)

## Method 2: Load data directly from AVRO files

For offline analysis, load AVRO files directly:

In [ ]:
# Load AVRO file from relay storage
# data = read_avro('/var/speeddata/avro/example_20250123.avro')
# print(data)

## Data Access

Roelle DataSet provides multiple convenient access patterns:

In [ ]:
# Access signal by channel.signal notation
voltage = data.example.voltage
current = data.example.current
time = data.example.time

print(f"Voltage shape: {voltage.shape}")
print(f"Current shape: {current.shape}")
print(f"Time points: {len(time)}")

## Visualization

Plot time-series data:

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Plot voltage
ax1.plot(time, voltage, 'b-', label='Voltage')
ax1.set_ylabel('Voltage (V)')
ax1.legend()
ax1.grid(True)

# Plot current
ax2.plot(time, current, 'r-', label='Current')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Current (A)')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Analysis

Perform basic statistical analysis:

In [ ]:
print("Voltage Statistics:")
print(f"  Mean: {np.mean(voltage):.3f} V")
print(f"  Std:  {np.std(voltage):.3f} V")
print(f"  Min:  {np.min(voltage):.3f} V")
print(f"  Max:  {np.max(voltage):.3f} V")

print("\nCurrent Statistics:")
print(f"  Mean: {np.mean(current):.3f} A")
print(f"  Std:  {np.std(current):.3f} A")
print(f"  Min:  {np.min(current):.3f} A")
print(f"  Max:  {np.max(current):.3f} A")

## Power Calculation

Calculate and plot instantaneous power:

In [ ]:
power = voltage * current

plt.figure(figsize=(12, 4))
plt.plot(time, power, 'g-', label='Power (V×I)')
plt.xlabel('Time (s)')
plt.ylabel('Power (W)')
plt.title(f'Instantaneous Power (Mean: {np.mean(power):.2f} W)')
plt.legend()
plt.grid(True)
plt.show()

print(f"Average Power: {np.mean(power):.3f} W")
print(f"Peak Power:    {np.max(power):.3f} W")